<a href="https://colab.research.google.com/github/BriML/People-count-YOLO8v2/blob/main/Fexco_Mercantil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicio (ejecutar primero)

In [1]:
import torch
print("Torch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())

# Instalaciones
#!pip install -q "ultralytics<=8.3.40" supervision
#!pip uninstall -y supervision
!pip install "ultralytics==8.3.40" "supervision==0.24.0"
#!pip install -U supervision ultralytics
!git clone -q https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -q -r requirements.txt
%cd ..

# Detectron2
!pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

from IPython.display import clear_output
clear_output()

# Importaciones (segunda ejecucion)

In [2]:
import torch
import supervision as sv
import detectron2
from ultralytics import YOLO
print("Torch CUDA disponible:", torch.cuda.is_available())
print("Supervision:", sv.__version__)
print("Detectron2:", detectron2.__version__)

Torch CUDA disponible: True
Supervision: 0.24.0
Detectron2: 0.6


# Modelos (tercera ejecucion)

In [3]:
# YOLOv8
yolo8_model = YOLO('yolov8s.pt')

# YOLOv5
yolo5_model = torch.hub.load('ultralytics/yolov5', 'yolov5x6', trust_repo=True)

# Detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-6 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients, 209.6 GFLOPs
Adding AutoShape... 
model_final_f10217.pkl: 178MB [00:00, 221MB/s]                           


# Utilizar videos directo de drive (cuarta ejecucion)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Quinta ejecucion

In [18]:
import os
video_dir = '/content/drive/MyDrive/Fexco-Proyecto-IA/prueba-videos/26-abril-2025/noche-20-hrs/cuartos-15-minutos'  #cambiar,la ruta de acuerdo a donde se encuentren los videos
video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])[:3]  # primeros 3 videos

# Antiguo

In [ ]:
import numpy as np
import supervision as sv

# initiate polygon zone
polygon = np.array([
          [1614, 272],
          [1900, 663],
          [1903, 1059],
          [29, 1063],
          [21, 427]
          ])
video_info = sv.VideoInfo.from_video_path('/content/mis_videos/prueba30fps.mp4')
zone = sv.PolygonZone(polygon=polygon)

# initiate annotators
box_annotator = sv.BoundingBoxAnnotator(thickness=4)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.GREEN, thickness=6, text_thickness=6, text_scale=4)

# extract video frame
generator = sv.get_video_frames_generator('/content/mis_videos/prueba30fps.mp4')
iterator = iter(generator)
frame = next(iterator)

# detect
outputs = predictor(frame)
detections = sv.Detections(
    xyxy=outputs["instances"].pred_boxes.tensor.cpu().numpy(),
    confidence=outputs["instances"].scores.cpu().numpy(),
    class_id=outputs["instances"].pred_classes.cpu().numpy().astype(int)
)
detections = detections[detections.class_id == 0]
zone.trigger(detections=detections)

# annotate
frame = box_annotator.annotate(scene=frame, detections=detections)
frame = zone_annotator.annotate(scene=frame)

%matplotlib inline
sv.plot_image(frame, (16, 16))

Exception: Could not open video at /content/mis_videos/prueba30fps.mp4

# Prueba x

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO

model = YOLO('yolov8s.pt')

def process_video(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    person_counts = []

    output_path = os.path.join(output_dir, os.path.basename(video_path))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (1920, 1080))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame)
        boxes = results[0].boxes
        person_count = sum(1 for cls in boxes.cls if int(cls) == 0)  # clase 0 es "persona" en COCO
        person_counts.append(person_count)

        annotated_frame = results[0].plot()
        out.write(annotated_frame)

        frame_count += 1

    cap.release()
    out.release()

    # Guardar CSV
    csv_path = os.path.splitext(output_path)[0] + '.csv'
    df = pd.DataFrame({'frame': list(range(len(person_counts))), 'count': person_counts})
    df['max'] = df['count'].max()
    df['min'] = df['count'].min()
    df.to_csv(csv_path, index=False)

    print(f"Procesado: {video_path} -> {output_path}")
    return output_path, csv_path


# Otras zonas

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import supervision as sv
from google.colab import drive

# Montar Drive
#drive.mount('/content/drive')

# Directorios
INPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/videos-pruebados'
OUTPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultados-prueba'
CSV_PATH = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultados-prueba/resultados_personas.csv'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Zonas poligonales (puedes modificarlas)
polygons = [
    np.array([[962, 281], [962, 402], [1209, 388], [1203, 263]], np.int32),  # zona de cuentas
    np.array([[1314, 267], [1247, 532], [1896, 642], [1917, 394], [1594, 243]], np.int32),  #zona pelotas

]

colors = sv.ColorPalette.DEFAULT
zones = [sv.PolygonZone(polygon=p) for p in polygons]
zone_annotators = [sv.PolygonZoneAnnotator(zone=z, color=colors.by_idx(i), thickness=3, text_thickness=2) for i, z in enumerate(zones)]
box_annotators = [sv.BoundingBoxAnnotator(color=colors.by_idx(i), thickness=2) for i in range(len(zones))]

# Modelo
from ultralytics import YOLO
model = YOLO("yolov8s.pt")

# Datos finales
resultados = []

# Procesar cada video
for nombre_video in os.listdir(INPUT_DIR):
    if not nombre_video.endswith('.mp4'):
        continue

    ruta_video = os.path.join(INPUT_DIR, nombre_video)
    cap = cv2.VideoCapture(ruta_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    writer = cv2.VideoWriter(os.path.join(OUTPUT_DIR, f"resultado_{nombre_video}"),
                             cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_idx = 0
    conteos_zona = [[] for _ in zones]

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]  # Solo personas

        for i, (zone, annotator_zone, annotator_box) in enumerate(zip(zones, zone_annotators, box_annotators)):
            mask = zone.trigger(detections)
            filtered = detections[mask]
            conteos_zona[i].append(len(filtered))
            frame = annotator_box.annotate(scene=frame, detections=filtered)
            frame = annotator_zone.annotate(scene=frame)

        writer.write(frame)

        frame_idx += 1

    cap.release()
    writer.release()

    # Guardar datos
    for i, conteo in enumerate(conteos_zona):
        if not conteo:
            continue
        resultados.append({
            "video": nombre_video,
            "zona": f"zona_{i+1}",
            "nro_personas": sum(conteo),
            "min": min(conteo),
            "max": max(conteo),
            "frame_min": conteo.index(min(conteo)),
            "frame_max": conteo.index(max(conteo))
        })

# Guardar CSV
df = pd.DataFrame(resultados)
df.to_csv(CSV_PATH, index=False)
print(f"CSV guardado en: {CSV_PATH}")


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


CSV guardado en: /content/drive/MyDrive/Fexco-Proyecto-IA/resultados-prueba/resultados_personas.csv


# Datos guardados (sexta ejecucion)

In [19]:
# Rutas, tanto input como output, cambiar de acuerdo a la ruta de entrada y la ruta de salida.
INPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/prueba-videos/26-abril-2025/noche-20-hrs/cuartos-15-minutos'
ESPECIFICAS_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultados-tres/zonas-de-interes'# ruta para salidas especificas
GENERAL_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/Resultados-oficiales/26-abril/cuartos-15/zona-stand'#ruta para salidas de la zona general
CSV_ESPECIFICAS = os.path.join(ESPECIFICAS_DIR, 'zonas_especificas.csv')
CSV_GENERAL = os.path.join(GENERAL_DIR, 'zona_general.csv')
os.makedirs(ESPECIFICAS_DIR, exist_ok=True)
os.makedirs(GENERAL_DIR, exist_ok=True)

# importaciones

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import supervision as sv

# Zona general

In [ ]:
# Zona general
polygons_general = [
    np.array([[1594, 239], [1917, 385], [1908, 1071], [17, 1059], [32, 378]], np.int32)
]

zones = [sv.PolygonZone(polygon=p) for p in polygons_general]
zone_annotators = [sv.PolygonZoneAnnotator(zone=z, color=colors.by_idx(0)) for z in zones]
box_annotators = [sv.BoundingBoxAnnotator(color=colors.by_idx(0))]

# Modelo
from ultralytics import YOLO
model = YOLO("yolov8s.pt")

resultados_general = []

for nombre_video in os.listdir(INPUT_DIR):
    if not nombre_video.endswith('.mp4'):
        continue

    ruta_video = os.path.join(INPUT_DIR, nombre_video)
    cap = cv2.VideoCapture(ruta_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))

    writer = cv2.VideoWriter(os.path.join(GENERAL_DIR, f"resultado_{nombre_video}"),
                             cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_idx = 0
    conteos_zona = [[]]

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]

        mask = zones[0].trigger(detections)
        filtered = detections[mask]
        conteos_zona[0].append(len(filtered))
        frame = box_annotators[0].annotate(scene=frame, detections=filtered)
        frame = zone_annotators[0].annotate(scene=frame)

        writer.write(frame)
        frame_idx += 1

    cap.release()
    writer.release()

    conteo = conteos_zona[0]
    if conteo:
        resultados_general.append({
            "video": nombre_video,
            "zona": "zona_general",
            "nro-maximo-personas-registradas-en-un-frame": max(conteo),
            "nro-personas-en-ultimo-frame": conteo[-1],
            "nro-minimo-personas-registradas-en-un-frame": min(conteo)
        })

# Guardar CSV
df_general = pd.DataFrame(resultados_general)
df_general.to_csv(CSV_GENERAL, index=False)
print(f"Zona general guardada en: {CSV_GENERAL}")

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Zona general guardada en: /content/drive/MyDrive/Fexco-Proyecto-IA/resultados-prueba2/zona-general/zona_general.csv


# Zonas especificas

In [ ]:
# Zonas específicas
polygons_especificos = [
    np.array([[962, 281], [962, 402], [1209, 388], [1203, 263]], np.int32),  # zona cuentas
    np.array([[1314, 267], [1247, 532], [1896, 642], [1917, 394], [1594, 243]], np.int32),  # zona pelotas
]

zones = [sv.PolygonZone(polygon=p) for p in polygons_especificos]
colors = sv.ColorPalette.DEFAULT
zone_annotators = [sv.PolygonZoneAnnotator(zone=z, color=colors.by_idx(i)) for i, z in enumerate(zones)]
box_annotators = [sv.BoundingBoxAnnotator(color=colors.by_idx(i)) for i in range(len(zones))]

# Modelo
from ultralytics import YOLO
model = YOLO("yolov8s.pt")

resultados = []

for nombre_video in os.listdir(INPUT_DIR):
    if not nombre_video.endswith('.mp4'):
        continue

    ruta_video = os.path.join(INPUT_DIR, nombre_video)
    cap = cv2.VideoCapture(ruta_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))

    writer = cv2.VideoWriter(os.path.join(ESPECIFICAS_DIR, f"anotado_{nombre_video}"),
                             cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    frame_idx = 0
    conteos_zona = [[] for _ in zones]

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]

        for i, (zone, annotator_zone, annotator_box) in enumerate(zip(zones, zone_annotators, box_annotators)):
            mask = zone.trigger(detections)
            filtered = detections[mask]
            conteos_zona[i].append(len(filtered))
            frame = annotator_box.annotate(scene=frame, detections=filtered)
            frame = annotator_zone.annotate(scene=frame)

        writer.write(frame)
        frame_idx += 1

    cap.release()
    writer.release()

    # Guardar datos por zona
    for i, conteo in enumerate(conteos_zona):
        if not conteo:
            continue
        resultados.append({
            "video": nombre_video,
            "zona": f"zona_especifica_{i+1}",
            "nro-maximo-personas-registradas-en-un-frame": max(conteo),
            "nro-personas-en-ultimo-frame": conteo[-1],
            "nro-minimo-personas-registradas-en-un-frame": min(conteo)
        })

# Guardar CSV
df = pd.DataFrame(resultados)
df.to_csv(CSV_ESPECIFICAS, index=False)
print(f"Zonas específicas guardadas en: {CSV_ESPECIFICAS}")

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Zonas específicas guardadas en: /content/drive/MyDrive/Fexco-Proyecto-IA/resultados-pruebas2/zonas-de-interes/zonas_especificas.csv


# Zonas general con bytetracking

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import supervision as sv
from ultralytics import YOLO
#from supervision.tools.detections import Detections
#from supervision.tracking.byte_track import ByteTrack

# Directorios
INPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/videos-pruebados'
OUTPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultados-tres/resultado-zona-general'
CSV_PATH = os.path.join(OUTPUT_DIR, "resultados_zona_general.csv")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Inicializar modelo y color
model = YOLO("yolov8s.pt")
color = sv.Color.BLUE#blue DEFAULT.by_idx(0)

tracker = sv.ByteTrack(track_activation_threshold=0.25,
    lost_track_buffer=30,
    minimum_matching_threshold=0.8,
    frame_rate=30,
    minimum_consecutive_frames=3)

resultados = []

for nombre_video in os.listdir(INPUT_DIR):
    if not nombre_video.endswith('.mp4'):
        continue

    print(f"Procesando (zona general): {nombre_video}")
    ruta_video = os.path.join(INPUT_DIR, nombre_video)
    cap = cv2.VideoCapture(ruta_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))

    writer = cv2.VideoWriter(
        os.path.join(OUTPUT_DIR, f"general_{nombre_video}"),
        cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))


    annotator_box = sv.BoundingBoxAnnotator(color=color)
    conteos = []
    visitantes_ids = set()

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]

        detections = tracker.update_with_detections(detections)

        conteos.append(len(detections))

        nuevos_ids = set(detections.tracker_id) - visitantes_ids
        visitantes_ids.update(nuevos_ids)

        frame = annotator_box.annotate(scene=frame, detections=detections)
        writer.write(frame)

    cap.release()
    writer.release()

    if conteos:
        resultados.append({
            "video": nombre_video,
            "zona": "zona_general",
            "nro_maximo_personas_en_un_frame": max(conteos),
            "nro_minimo_personas_en_un_frame": min(conteos),
            "nro_personas_ultimo_frame": conteos[-1],
            "nro_nuevos_visitantes": len(visitantes_ids),
            "promedio_personas_por_frame": round(sum(conteos) / len(conteos), 2)
        })

df = pd.DataFrame(resultados)
df.to_csv(CSV_PATH, index=False)
print(f"CSV guardado en: {CSV_PATH}")

Procesando (zona general): prueba-corta.mp4


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Procesando (zona general): prueba20fps.mp4


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Procesando (zona general): video-test5min.mp4


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


CSV guardado en: /content/drive/MyDrive/Fexco-Proyecto-IA/resultados-tres/resultado-zona-general/resultados_zona_general.csv


# Zona general con LineZone

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import supervision as sv
from ultralytics import YOLO

# Directorios
INPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/videos-pruebados'
OUTPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultado-cuatro/resultado-zona-general'
CSV_PATH = os.path.join(OUTPUT_DIR, "resultados_zona_general.csv")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Inicializar modelo y tracker
model = YOLO("yolov8s.pt")
color = sv.Color.BLUE

tracker = sv.ByteTrack(
    track_activation_threshold=0.25,
    lost_track_buffer=30,
    minimum_matching_threshold=0.8,
    frame_rate=20,
    minimum_consecutive_frames=3
)

resultados = []

for nombre_video in os.listdir(INPUT_DIR):
    if not nombre_video.endswith('.mp4'):
        continue

    print(f"Procesando (zona general): {nombre_video}")
    ruta_video = os.path.join(INPUT_DIR, nombre_video)
    cap = cv2.VideoCapture(ruta_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))

    writer = cv2.VideoWriter(
        os.path.join(OUTPUT_DIR, f"general_{nombre_video}"),
        cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height)
    )

    annotator_box = sv.BoundingBoxAnnotator(color=color)

    # DEFINIR LA LÍNEA VIRTUAL DE ENTRADA (ajusta las coordenadas según tu video)
    # Por ejemplo, una línea horizontal en la parte inferior del frame
    line_start = sv.Point(100, height - 50)
    line_end = sv.Point(width - 100, height - 50)
    line_zone = sv.LineZone(start=line_start, end=line_end)
    line_annotator = sv.LineZoneAnnotator(thickness=3, text_thickness=2, text_scale=1)

    visitantes_ids = set()
    conteos = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]

        detections = tracker.update_with_detections(detections)

        # 🔹 ACTUALIZAR LA ZONA DE LÍNEA con los nuevos detections
        line_zone.trigger(detections=detections)

        # 🔹 NUEVOS track_ids que cruzaron la línea en dirección válida
        nuevos_ids = line_zone.triggered_ids - visitantes_ids
        visitantes_ids.update(nuevos_ids)

        # Anotaciones visuales
        frame = annotator_box.annotate(scene=frame, detections=detections)
        frame = line_annotator.annotate(scene=frame, line_counter=line_zone)

        # Conteo por frame (puede servir para estadística)
        conteos.append(len(detections))
        writer.write(frame)

    cap.release()
    writer.release()

    if conteos:
        resultados.append({
            "video": nombre_video,
            "zona": "zona_general",
            "nro_maximo_personas_en_un_frame": max(conteos),
            "nro_minimo_personas_en_un_frame": min(conteos),
            "nro_personas_ultimo_frame": conteos[-1],
            "nro_nuevos_visitantes": len(visitantes_ids),
            "promedio_personas_por_frame": round(sum(conteos) / len(conteos), 2)
        })

# Exportar resultados
df = pd.DataFrame(resultados)
df.to_csv(CSV_PATH, index=False)
print(f"CSV guardado en: {CSV_PATH}")

# LineZone con puerta de entrada

In [20]:
import cv2
import numpy as np
import pandas as pd
import os
import supervision as sv
from ultralytics import YOLO

# Configuración
INPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/prueba-videos/26-abril-2025/noche-20-hrs/cuartos-15-minutos'
OUTPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/Resultados-oficiales/26-abril/cuartos-15/zona-stand'
CSV_PATH = os.path.join(OUTPUT_DIR, "resultados_zona_general.csv")
os.makedirs(OUTPUT_DIR, exist_ok=True)

model = YOLO("yolov8s.pt")
color = sv.Color.WHITE

tracker = sv.ByteTrack(track_activation_threshold=0.25,
    lost_track_buffer=20,
    minimum_matching_threshold=0.8,
    frame_rate=20,
    minimum_consecutive_frames=3)

resultados = []

# Línea vertical a la derecha (puerta)
LINE_X = 1800
LINE_START = sv.Point(LINE_X, 200)
LINE_END = sv.Point(LINE_X, 900)
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)
line_annotator = sv.LineZoneAnnotator(thickness=3, text_thickness=2, text_scale=1)

for nombre_video in os.listdir(INPUT_DIR):
    if not nombre_video.endswith('.mp4'):
        continue

    print(f"Procesando: {nombre_video}")
    ruta_video = os.path.join(INPUT_DIR, nombre_video)
    cap = cv2.VideoCapture(ruta_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width, height = int(cap.get(3)), int(cap.get(4))

    writer = cv2.VideoWriter(
        os.path.join(OUTPUT_DIR, f"general_{nombre_video}"),
        cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height)
    )

    annotator_box = sv.BoundingBoxAnnotator(color=color)
    previous_positions = {}
    visitantes_ids = set()
    conteos = []

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Detección y filtrado de personas
        results = model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]
        detections = tracker.update_with_detections(detections)

        # Conteo de visitantes cruzando la línea
        for i in range(len(detections)):
            track_id = detections.tracker_id[i]
            x1, y1, x2, y2 = detections.xyxy[i]
            center_x = (x1 + x2) / 2

            if track_id in previous_positions:
                prev_x = previous_positions[track_id]

                # Si cruzó de derecha a izquierda por la línea de entrada
                if prev_x > LINE_X and center_x <= LINE_X and track_id not in visitantes_ids:
                    visitantes_ids.add(track_id)

            previous_positions[track_id] = center_x

        # Anotar detecciones y línea
        frame = annotator_box.annotate(scene=frame, detections=detections)
        #frame = line_annotator.annotate(frame, line_counter=line_zone)
        cv2.line(frame, (LINE_START.x, LINE_START.y), (LINE_END.x, LINE_END.y), (255, 255, 255), thickness=3)

        conteos.append(len(detections))
        writer.write(frame)

    cap.release()
    writer.release()

    # Guardar datos del video
    if conteos:
        resultados.append({
            "video": nombre_video,
            "zona": "zona_stand",
            "nro_maximo_personas_en_un_frame": max(conteos),
            "nro_personas_ultimo_frame": conteos[-1],
            "nro_nuevos_visitantes": len(visitantes_ids),
            "promedio_personas_por_frame": round(sum(conteos) / len(conteos))
        })

# Exportar resultados
df = pd.DataFrame(resultados)
df.to_csv(CSV_PATH, index=False)
print(f"CSV guardado en: {CSV_PATH}")

Procesando: 26-04-hrs-20.45.00-20.50.00.mp4


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Procesando: 26-04-hrs-20.50.00-20.55.00.mp4


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Procesando: 26-04-hrs-20.55.00-21.00.00.mp4


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


CSV guardado en: /content/drive/MyDrive/Fexco-Proyecto-IA/Resultados-oficiales/26-abril/cuartos-15/zona-stand/resultados_zona_general.csv


# Prueba de heatmaps (no ejecutar)

In [ ]:
from supervision.annotators import HeatMapAnnotator

INPUT_DIR = '/content/drive/MyDrive/Fexco-Proyecto-IA/prueba-videos'
# Configurar salida
OUTPUT_VIDEO = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultado-cuatro/mapas-de-calor'
os.makedirs(os.path.dirname(OUTPUT_VIDEO), exist_ok=True)

for nombre_video in os.listdir(INPUT_DIR):
    if not nombre_video.endswith('.mp4'):
        continue
        cap = cv2.VideoCapture()
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width, height = int(cap.get(3)), int(cap.get(4))

writer = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

model = YOLO("yolov8s.pt")
tracker = ByteTrack()
heatmap_annotator = HeatMapAnnotator(
    opacity=0.5,  # transparencia de la superposición (0 a 1)
    kernel_size=80,  # radio de "influencia" de cada punto detectado
    colormap=cv2.COLORMAP_JET  # puedes cambiar a HOT, VIRIDIS, etc.
)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    results = model(frame, verbose=False)[0]
    detections = Detections.from_ultralytics(results)
    detections = detections[detections.class_id == 0]

    detections = tracker.update_with_detections(detections)

    # Anotar heatmap con el centro de los bounding boxes
    heatmap_annotator.update(detections.xyxy, frame_shape=frame.shape)
    frame = heatmap_annotator.annotate(scene=frame)

    writer.write(frame)

cap.release()
writer.release()
print("Video con heatmap generado correctamente.")

# Heatmap sin **bytetrack**

In [ ]:
import os
import cv2
import supervision as sv
from ultralytics import YOLO

video_dir = '/content/drive/MyDrive/Fexco-Proyecto-IA/prueba-videos'
video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])[:3]

# Inicializar modelo y anotador
model = YOLO('yolov8s.pt')
heatmap_annotator = sv.HeatMapAnnotator()

# Usamos el primer video para obtener la configuración
primer_video_path = os.path.join(video_dir, video_files[0])
video_info = sv.VideoInfo.from_video_path(video_path=primer_video_path)

# Ruta de salida del video heatmap
output_path = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultado-cuatro/mapas-de-calor/heatmap_resultado.mp4'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Crear el generador de frames personalizado
def get_video_frames_generator(source_path):
    cap = cv2.VideoCapture(source_path)
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        yield frame
    cap.release()

# Abrimos el archivo de salida para escritura
with sv.VideoSink(target_path=output_path, video_info=video_info) as sink:
    for video_name in video_files:
        video_path = os.path.join(video_dir, video_name)
        for frame in get_video_frames_generator(video_path):
            result = model(frame)[0]
            detections = sv.Detections.from_ultralytics(result)
            detections = detections[detections.class_id == 0]  # solo personas
            annotated_frame = heatmap_annotator.annotate(
                scene=frame.copy(),
                detections=detections
            )
            sink.write_frame(frame=annotated_frame)

print("Video de heatmap generado correctamente.")

Se han truncado las últimas 5000 líneas del flujo de salida.
Speed: 7.8ms preprocess, 11.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 3 suitcases, 11.1ms
Speed: 3.7ms preprocess, 11.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 2 suitcases, 12.2ms
Speed: 4.2ms preprocess, 12.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 suitcase, 21.4ms
Speed: 6.1ms preprocess, 21.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 suitcase, 15.9ms
Speed: 4.4ms preprocess, 15.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 1 suitcase, 14.5ms
Speed: 3.5ms preprocess, 14.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 12.7ms
Speed: 4.2ms preprocess, 12.7ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 

# Heatmap final

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from supervision import Detections
from tqdm import tqdm

# Ruta a tus videos
video_dir = '/content/drive/MyDrive/Fexco-Proyecto-IA/prueba-videos'
video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])[:3]
video_paths = [os.path.join(video_dir, f) for f in video_files]

# Cargar modelo YOLO (puedes cambiar a yolov8n.pt si quieres más velocidad)
model = YOLO('yolov8s.pt')

# Inicializar heatmap acumulador
heatmap_accum = None
frame_base = None
max_detections = 0

# Recorrer los videos
for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # Detección de personas
        results = model(frame, verbose=False)[0]
        detections = Detections.from_ultralytics(results)
        detections = detections[detections.class_id == 0]  # Solo clase persona

        # Crear máscara vacía para este frame
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        for xyxy in detections.xyxy:
            x1, y1, x2, y2 = map(int, xyxy)
            cv2.rectangle(mask, (x1, y1), (x2, y2), 1, thickness=-1)

        # Acumular en el heatmap
        if heatmap_accum is None:
            heatmap_accum = np.zeros_like(mask, dtype=np.float32)
        heatmap_accum += mask

        # Elegir como base el frame con más personas
        if len(detections) > max_detections:
            max_detections = len(detections)
            frame_base = frame.copy()

    cap.release()

# Normalizar heatmap (0-255) y aplicar colormap
heatmap_norm = cv2.normalize(heatmap_accum, None, 0, 255, cv2.NORM_MINMAX)
heatmap_uint8 = heatmap_norm.astype(np.uint8)
colored_heatmap = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)

# Redimensionar base si difiere por errores de lectura
if frame_base.shape[:2] != colored_heatmap.shape[:2]:
    frame_base = cv2.resize(frame_base, (colored_heatmap.shape[1], colored_heatmap.shape[0]))

# Mezclar: 90% heatmap, 10% imagen original
final_overlay = cv2.addWeighted(colored_heatmap, 0.9, frame_base, 0.1, 0)

# Guardar resultado
output_path = '/content/drive/MyDrive/Fexco-Proyecto-IA/resultado-heatmap-unico/final_heatmap.jpg'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
cv2.imwrite(output_path, final_overlay)

print(f"Imagen de heatmap generada correctamente: {output_path}")
